# Golden Filter for Video 

## 1、加载视频文件并展示

In [27]:
import cv2
import numpy as np
import time

ModuleNotFoundError: No module named 'cv2'

加载目录下的视频文件test.mp4，播放未经处理的原视频。

In [29]:
from IPython.display import Video
Video("./test.mp4")

输出视频的各类信息。

In [21]:
vc = cv2.VideoCapture("./test.mp4")

width = int(vc.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vc.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"视频尺寸: {width} x {height}")

fps = int(vc.get(cv2.CAP_PROP_FPS))
print("视频帧率:", fps)

frame_c = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
print("视频帧数:", frame_c)

NameError: name 'cv2' is not defined

## 2、使用python函数为视频添加怀旧滤镜

    通过设置适当的线性核对视频每一帧的像素BGR值进行线性变换处理，以达到添加怀旧滤镜的效果。
    处理时间可能较慢，请耐心等待。

In [7]:
#设置输出格式和输出流
fourcc = cv2.VideoWriter_fourcc(*'XVID') 
output = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height), True)

total_time = 0

#滤镜参数
kernel = np.array([[0.131, 0.534, 0.272],
                  [0.168, 0.686, 0.349],
                  [0.189, 0.769, 0.393]])
#R、G、B分量的点运算映射函数分别为
#R = 0.393r+0.769g+0.189b
#G = 0.349r+0.686g+0.168b
#B = 0.272r+0.534g+0.131b

while(vc.isOpened()):
    
    ret, frame = vc.read()
    if ret==True:
        start_time = time.perf_counter()
        #每读一帧，对图像处理并写入文件中
        frame = cv2.transform(frame, kernel)
        
        end_time = time.perf_counter()
        total_time += end_time - start_time
        
        output.write(frame)
    else:
       break
 
print("处理耗时：{}s".format(total_time))

vc.release()
output.release()

处理耗时：22.124585161999676s


In [30]:
#展示处理后的视频
Video("./output.mp4")

## 3、使用硬件加速视频处理

In [9]:
#重新加载视频
vc = cv2.VideoCapture("./test.mp4")

width = int(vc.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vc.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(vc.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output2 = cv2.VideoWriter('output2.mp4', fourcc, fps, (width, height), True)

In [10]:
from pynq import Overlay, allocate

#overlay加载
overlay = Overlay("./overlay_golden/golden_transfer.bit")

golden_filter = overlay.golden_transfer_0
#写入参数
golden_filter.write(0x10, height)
golden_filter.write(0x18, width)

#启动IP，提取dma
dma = overlay.axi_dma_0

In [11]:
block_size = 24 #一次读入的数据块大小
pad_width = (block_size - 1) - (height * width * 3 - 1) % block_size #根据视频帧的尺寸对缓冲区大小进行填充

#分配缓冲区空间
input_buffer = allocate(shape=(width * height * 3 + pad_width,), dtype='u1')
output_buffer = allocate(shape=(width * height * 3 + pad_width,), dtype='u1')

In [12]:
total_time = 0

while(vc.isOpened()):
    #循环读入视频帧
    ret, frame = vc.read()
    if ret==True:
        #对帧进行填充并放置到输入缓冲区中
        frame = np.pad(frame.flatten(), (0, pad_width), mode='constant', constant_values=0)
        np.copyto(input_buffer, frame)
        
        golden_filter.register_map.CTRL.AP_START = 1
        
        start_time = time.perf_counter()
        #使用硬件对视频帧进行处理
        #调用DMA读取输入缓冲，并将数据发送到AXI Stream Master
        #之后，DMA应从AXI Stream Slave中将结果写回到输出缓冲中
        dma.sendchannel.transfer(input_buffer)
        dma.recvchannel.transfer(output_buffer)
        dma.sendchannel.wait() #'wait'语句确保了DMA的处理操作已经完成
        dma.recvchannel.wait()
        
        end_time = time.perf_counter()
        total_time += end_time - start_time
        #将处理后的视频帧输出到文件
        output2.write((np.resize(output_buffer, (height, width, 3))).reshape(height, width, 3))

    else:
        break
        

print("处理耗时：{}s".format(total_time))

vc.release()
output2.release()

处理耗时：1.6548340710000957s


展示处理得到的视频：

In [31]:
Video("./output2.mp4")

## 4、创建用户接口

In [14]:
def GoldenFilter(f, input_path, output_path):   #输入overlay, 要处理的视频目录和结果存放目录
               
    vc = cv2.VideoCapture(input_path)

    width = int(vc.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vc.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(vc.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    output3 = cv2.VideoWriter(output_path, fourcc, fps, (width, height), True)

    f.write(0x10, height)
    f.write(0x18, width)

    block_size = 24
    pad_width = (block_size - 1) - (height * width * 3 - 1) % block_size

    input_buffer = allocate(shape=(width * height * 3 + pad_width,), dtype='u1')
    output_buffer = allocate(shape=(width * height * 3 + pad_width,), dtype='u1')

    while(vc.isOpened()):
    
        ret, frame = vc.read()
        if ret==True:

            frame = np.pad(frame.flatten(), (0, pad_width), mode='constant', constant_values=0)
            np.copyto(input_buffer, frame)
        
            f.register_map.CTRL.AP_START = 1
        
            dma.sendchannel.transfer(input_buffer)
            dma.recvchannel.transfer(output_buffer)
            dma.sendchannel.wait()
            dma.recvchannel.wait()
        
            output3.write((np.resize(output_buffer, (height, width, 3))).reshape(height, width, 3))

        else:
            break
            
    vc.release()
    output3.release()
        
    return

直接调用上方描述的方法并输出结果。

In [33]:
GoldenFilter(golden_filter, './test.mp4', './output3.mp4')
from IPython.display import Video
Video('./output3.mp4')